# WeatherWise – Starter Notebook
This notebook contains code for a weather advisor app including data fetching, visualization, and a simple UI.

In [ ]:

# Install required packages (uncomment if needed)
# !pip install fetch-my-weather
# !pip install hands-on-ai

import os
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response


In [ ]:

from datetime import datetime
import pprint

OPENWEATHER_API_KEY = "aabb92d590c9b7c0c5741eae5c5768f7"
LOCATION = "Perth,AU"
UNITS = "metric"
FORECAST_DAYS = 5

def degrees_to_compass(degrees):
    directions = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE",
                  "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
    return directions[round(degrees / 22.5) % 16]

def get_weather_forecast():
    try:
        url = f"http://api.openweathermap.org/data/2.5/forecast?q={LOCATION}&appid={OPENWEATHER_API_KEY}&units={UNITS}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        forecast = []
        current_date = None
        daily_data = {}
        for entry in data["list"]:
            entry_date = entry["dt_txt"].split()[0]
            if entry_date != current_date:
                if current_date is not None:
                    forecast.append(daily_data)
                current_date = entry_date
                daily_data = {
                    "date": current_date,
                    "day_of_week": datetime.strptime(current_date, "%Y-%m-%d").strftime("%A"),
                    "times": [],
                    "max_temp": -float('inf'),
                    "min_temp": float('inf'),
                    "conditions": set()
                }
            time_data = {
                "time": entry["dt_txt"].split()[1][:5],
                "temperature": round(entry["main"]["temp"], 1),
                "feels_like": round(entry["main"]["feels_like"], 1),
                "condition": entry["weather"][0]["description"].title(),
                "humidity": entry["main"]["humidity"],
                "wind_speed": round(entry["wind"]["speed"], 1),
                "wind_direction": degrees_to_compass(entry["wind"]["deg"]),
                "precipitation_chance": round(entry.get("pop", 0) * 100),
                "pressure": entry["main"]["pressure"],
                "cloud_coverage": entry["clouds"]["all"],
                "visibility": entry.get("visibility", "N/A")
            }
            daily_data["times"].append(time_data)
            daily_data["max_temp"] = max(daily_data["max_temp"], time_data["temperature"])
            daily_data["min_temp"] = min(daily_data["min_temp"], time_data["temperature"])
            daily_data["conditions"].add(time_data["condition"])
        if current_date and daily_data:
            forecast.append(daily_data)
        return {
            "location": f"{data['city']['name']}, {data['city']['country']}",
            "latitude": data["city"]["coord"]["lat"],
            "longitude": data["city"]["coord"]["lon"],
            "forecast_days": min(FORECAST_DAYS, len(forecast)),
            "forecast": forecast[:FORECAST_DAYS],
            "units": {
                "temperature": "°C",
                "humidity": "%",
                "wind_speed": "m/s",
                "precipitation": "% chance",
                "pressure": "hPa",
                "visibility": "meters"
            },
            "api_source": "OpenWeatherMap",
            "generated_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    except requests.exceptions.RequestException as e:
        return {"error": f"API request failed: {str(e)}"}
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}

def display_forecast(weather_data):
    if "error" in weather_data:
        print(f"  Error: {weather_data['error']}")
        return
    print(f"\n  Weather Forecast for {weather_data['location']}  ")
    print(f"  Coordinates: {weather_data['latitude']}°N, {weather_data['longitude']}°E")
    print(f"  Generated on: {weather_data['generated_on']}")
    print("="*50)
    for day in weather_data["forecast"]:
        print(f"\n  {day['day_of_week']}, {day['date']}")
        print(f"  Temp: {day['min_temp']}°C to {day['max_temp']}°C")
        print(f"  Conditions: {', '.join(day['conditions'])}")
        print("-"*40)
        for time in day["times"]:
            print(f"  {time['time']}: {time['condition']}")
            print(f"    Temp: {time['temperature']}°C (Feels like {time['feels_like']}°C)")
            print(f"    Humidity: {time['humidity']}%")
            print(f"    Wind: {time['wind_speed']} m/s {time['wind_direction']}")
            print(f"    Rain chance: {time['precipitation_chance']}%")
            print(f"    Clouds: {time['cloud_coverage']}%")
            print("-"*20)

if __name__ == "__main__":
    print("Fetching weather data...")
    weather_data = get_weather_forecast()
    display_forecast(weather_data)
